In [1]:
# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys

# identify user's home directory
user_home_dir = expanduser('~')

# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco
from ecco_v4_py import vector_calc, scalar_calc

import gsw
import botocore  
import boto3
from boto3.session import Session
import cmocean
from collections import Counter
from dask.distributed import Client
import datetime
import fsspec
from gc import get_referents
import json
import numpy as np
from pathlib import Path
from pprint import pprint
import requests
import s3fs
import sys
from sys import getsizeof
import time as time
from types import ModuleType, FunctionType
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import zarr

In [2]:
ecco_v4r5_mon_mean_native_dir = Path('/efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/')

# list sub-directories (one per dataset)
ecco_v4r5_mon_mean_native_dataset_paths = np.sort(list(ecco_v4r5_mon_mean_native_dir.glob('*')))
# Select a dataset (the one containing temperature and salinity)
dataset_num = 20

print('selected ', ecco_v4r5_mon_mean_native_dataset_paths[dataset_num])
TS_dataset_dir = ecco_v4r5_mon_mean_native_dataset_paths[dataset_num]

# make a list of all of the files in the directory
TS_dataset_files = np.sort(list(TS_dataset_dir.glob('*nc')))

selected  /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_TEMPERATURE_SALINITY


In [3]:
from dask.distributed import Client

#  connec to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:44307") #find within Scheduler Address with orange red squares)
client.ncores
client.restart()

In [4]:
start_time = time.time();
ds_TS = None

files_to_load = TS_dataset_files

print(f'lazy-loading {len(files_to_load)} granules')
# first lazy load
ds_TS = xr.open_mfdataset(TS_dataset_files, 
                          parallel=True, data_vars='minimal',\
                          coords='minimal',compat='override',
                          combine='nested', concat_dim='time',
                          chunks={'time':12, 'tile':13,' k':50,'j':90,'i':90})

lazy-loading 336 granules


In [5]:
abs_sal = ds_TS.SALT
cons_temp = gsw.conversions.CT_from_pt(abs_sal, ds_TS.THETA)
sigma_2 = gsw.density.sigma2(abs_sal, cons_temp)
sigma_2 = sigma_2.to_dataset(name = 'SIG2')
sigma_2

<xarray.Dataset> Size: 14GB
Dimensions:  (i: 90, j: 90, k: 50, tile: 13, time: 336)
Coordinates:
  * i        (i) int32 360B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
  * j        (j) int32 360B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
  * k        (k) int32 200B 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
  * tile     (tile) int32 52B 0 1 2 3 4 5 6 7 8 9 10 11 12
  * time     (time) datetime64[ns] 3kB 1992-01-16T18:00:00 ... 2019-12-16
    XC       (tile, j, i) float32 421kB dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    YC       (tile, j, i) float32 421kB dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    Z        (k) float32 200B dask.array<chunksize=(50,), meta=np.ndarray>
Data variables:
    SIG2     (time, k, tile, j, i) float64 14GB dask.array<chunksize=(1, 25, 13, 90, 90), meta=np.ndarray>

In [ ]:
sigma_2.to_netcdf('Sigma_2_field.nc')